In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
def year_parser(months):
    years = []
    for i in range(len(months)):
        years.append(int(months[i][:4]))
    return years

def quarter_parser(months):
    qrtrs = []
    for i in range(len(months)):
        q = int(months[i][5:7])
        if q in [1, 2, 3]:
            qrtrs.append(1)
        elif q in [4, 5, 6]:
            qrtrs.append(2)
        elif q in [7, 8, 9]:
            qrtrs.append(3)
        elif q in [10, 11, 12]:
            qrtrs.append(4)
    return qrtrs

def m_parser(months):
    m = []
    for i in range(len(months)):
        m.append(int(months[i][5:7]))
    return m

In [3]:
train = pd.DataFrame(pd.read_csv('price_train.tsv',delimiter='\t',encoding='utf-8'))
test = pd.DataFrame(pd.read_csv('price_test.tsv',delimiter='\t',encoding='utf-8'))
hb = pd.DataFrame(pd.read_csv('price_housebase.tsv',delimiter='\t',encoding='utf-8'))
# sample_submission = pd.DataFrame(pd.read_csv('price_sample_submission.tsv',delimiter='\t',encoding='utf-8'))

train0 = pd.DataFrame(pd.read_csv('price_train.tsv',delimiter='\t',encoding='utf-8'))
test0 = pd.DataFrame(pd.read_csv('price_test.tsv',delimiter='\t',encoding='utf-8'))

In [4]:
hb = hb.dropna(axis=0)
hb

,beauty_cnt,building_id,building_series_id,shopping_cnt,flats_count,latitude,building_type,cafe_restaurant_eating_out_cnt,unified_address,entertainment_cnt,...,art_cnt,site_id,healthcare_cnt,expect_demolition,longitude,ceiling_height,has_elevator,laundry_and_repair_services_cnt,build_year,city_quadkey
0,36.0,3031030817531317628,0,45.0,0,55.705219,BRICK,24.0,"Россия, Москва, 1-й Автозаводский проезд, 3",18.0,...,7.0,0,16.0,False,37.656994,0.00,True,38.0,1937,120310101302011
1,36.0,3031030817531317630,0,45.0,0,55.704845,BRICK,24.0,"Россия, Москва, 1-й Автозаводский проезд, 5",18.0,...,7.0,0,16.0,False,37.657295,0.00,False,38.0,1962,120310101302011
2,36.0,1764947998512909470,0,45.0,0,55.704697,UNKNOWN,24.0,"Россия, Москва, 1-й Автозаводский проезд, 7/1",18.0,...,7.0,0,16.0,False,37.658203,0.00,False,38.0,0,120310101302011
3,6.0,2614890160181791580,1564812,23.0,176,55.811642,BRICK,11.0,"Россия, Москва, 1-й Амбулаторный проезд, 2",16.0,...,4.0,0,14.0,False,37.535599,2.65,True,25.0,1967,120310101031222
4,6.0,4152249910202725078,1564812,23.0,60,55.810810,BRICK,11.0,"Россия, Москва, 1-й Амбулаторный проезд, 5к1",16.0,...,4.0,0,14.0,False,37.534649,0.00,False,25.0,1959,120310101031222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112198,1.0,7404865526597723423,0,2.0,0,55.542339,UNKNOWN,1.0,"Россия, Московская область, Раменский городско...",4.0,...,1.0,0,2.0,False,38.311253,2.50,False,3.0,0,120310112011131
112199,0.0,6317574577050602872,0,0.0,0,55.267395,BRICK,0.0,"Россия, Московская область, Раменский городско...",1.0,...,0.0,0,1.0,False,38.470856,0.00,False,0.0,0,120310112303321
112200,41.0,6727800737206060131,0,32.0,0,55.734367,UNKNOWN,35.0,"пер. Щетининский, вл. 4, стр. 2, 3",12.0,...,34.0,0,32.0,False,37.622253,3.30,False,39.0,0,120310101300022
112201,14.0,4244359392749537526,1564812,12.0,0,55.681206,MONOLIT,5.0,"ул. Вавилова, 69А",21.0,...,4.0,0,4.0,False,37.551422,3.00,True,13.0,2018,120310101213210


In [5]:
hb.columns
hb.dtypes
#set(hb['building_type'])

beauty_cnt                          float64
building_id                           int64
building_series_id                    int64
shopping_cnt                        float64
flats_count                           int64
latitude                            float64
building_type                        object
cafe_restaurant_eating_out_cnt      float64
unified_address                      object
entertainment_cnt                   float64
sport_cnt                           float64
chain_cnt                           float64
groceries_and_everyday_items_cnt    float64
art_cnt                             float64
site_id                               int64
healthcare_cnt                      float64
expect_demolition                      bool
longitude                           float64
ceiling_height                      float64
has_elevator                           bool
laundry_and_repair_services_cnt     float64
build_year                            int64
city_quadkey                    

In [6]:
train.columns
train.dtypes

apartment_ratio                        float64
offers_in_sites_ratio                  float64
unique_active_buildings_count            int64
avg_price_sqm                          float64
month                                   object
unique_active_building_series_count      int64
rooms_offered_3_ratio                  float64
city_quadkey                             int64
median_price_sqm                       float64
rooms_offered_4_more_ratio             float64
rooms_offered_1_ratio                  float64
total_area_avg                         float64
studio_ratio                           float64
offer_count                              int64
rooms_offered_2_ratio                  float64
dtype: object

In [7]:
y_train = train['avg_price_sqm']
train = train.drop(columns=['avg_price_sqm', 'median_price_sqm'])
train

,apartment_ratio,offers_in_sites_ratio,unique_active_buildings_count,month,unique_active_building_series_count,rooms_offered_3_ratio,city_quadkey,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,offer_count,rooms_offered_2_ratio
0,0.0,0.0,1,2018-02-01,1,0.000000,120132230303223,0.0,0.000000,44.000000,0.0,1,1.000000
1,0.0,0.0,1,2018-03-01,1,0.000000,120132230303223,0.0,0.000000,44.000000,0.0,1,1.000000
2,0.0,0.0,1,2018-06-01,1,0.000000,120132230332230,0.0,1.000000,34.500000,0.0,1,0.000000
3,0.0,0.0,1,2018-07-01,1,0.000000,120132230332230,0.0,1.000000,34.500000,0.0,1,0.000000
4,0.0,0.0,1,2017-01-01,1,1.000000,120132231322113,0.0,0.000000,56.000000,0.0,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166022,0.0,0.0,9,2019-04-01,1,0.052632,120311020100310,0.0,0.052632,46.421053,0.0,19,0.894737
166023,0.0,0.0,9,2019-05-01,1,0.047619,120311020100310,0.0,0.047619,46.309524,0.0,21,0.904762
166024,0.0,0.0,9,2019-06-01,1,0.043478,120311020100310,0.0,0.043478,45.995652,0.0,23,0.913043
166025,0.0,0.0,10,2019-07-01,1,0.041667,120311020100310,0.0,0.041667,45.933333,0.0,24,0.916667


In [8]:
test

,month,city_quadkey
0,2020-03-01,120132230323210
1,2019-09-01,120132231322113
2,2019-10-01,120132231322113
3,2019-11-01,120132231322113
4,2019-12-01,120132231322113
...,...,...
38025,2019-11-01,120311020100310
38026,2019-12-01,120311020100310
38027,2020-01-01,120311020100310
38028,2020-02-01,120311020100310


In [9]:
len(set(test['city_quadkey']))

6121

In [10]:
from datetime import datetime

def month_parser(month):
    for i in tqdm(range(len(month))):
        month[i] = datetime.fromisoformat(month[i]).timestamp()

In [11]:
month_parser(test['month'])
month_parser(train['month'])

test['month'] = test['month'].astype('float64')
train['month'] = train['month'].astype('float64')

  0%|          | 0/38030 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 166027/166027 [06:53<00:00, 401.83it/s]


In [12]:
hb_avg = hb.groupby(['city_quadkey'], as_index=False).mean()

# SAMPLE SUBM 1

In [13]:
train0.corr()

,apartment_ratio,offers_in_sites_ratio,unique_active_buildings_count,avg_price_sqm,unique_active_building_series_count,rooms_offered_3_ratio,city_quadkey,median_price_sqm,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,offer_count,rooms_offered_2_ratio
apartment_ratio,1.000000,NaN,0.013678,0.156457,0.013749,-0.016691,0.025643,0.161424,0.047401,0.002993,0.078252,0.080328,0.106049,-0.015544
offers_in_sites_ratio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unique_active_buildings_count,0.013678,NaN,1.000000,0.350379,0.594614,0.033906,0.139391,0.339490,0.098307,-0.029549,0.100639,-0.032916,0.571821,-0.048896
avg_price_sqm,0.156457,NaN,0.350379,1.000000,0.379932,0.093827,0.219365,0.989666,0.415100,-0.162606,0.526918,0.003011,0.381294,-0.135692
unique_active_building_series_count,0.013749,NaN,0.594614,0.379932,1.000000,0.072186,0.206167,0.392844,0.058578,-0.038341,0.069095,-0.029766,0.306061,-0.053585
rooms_offered_3_ratio,-0.016691,NaN,0.033906,0.093827,0.072186,1.000000,0.046534,0.095404,-0.048640,-0.419526,0.222721,-0.093713,0.019773,-0.435645
city_quadkey,0.025643,NaN,0.139391,0.219365,0.206167,0.046534,1.000000,0.226143,0.054445,-0.033570,0.095296,-0.010826,0.099625,-0.035439
median_price_sqm,0.161424,NaN,0.339490,0.989666,0.392844,0.095404,0.226143,1.000000,0.401224,-0.158112,0.517569,0.003834,0.374174,-0.134491
rooms_offered_4_more_ratio,0.047401,NaN,0.098307,0.415100,0.058578,-0.048640,0.054445,0.401224,1.000000,-0.239874,0.693600,-0.011369,0.135928,-0.229233
rooms_offered_1_ratio,0.002993,NaN,-0.029549,-0.162606,-0.038341,-0.419526,-0.033570,-0.158112,-0.239874,1.000000,-0.373399,0.166694,-0.023029,-0.497417


In [14]:
###
# train = train.drop(columns=['apartment_ratio', 'studio_ratio', 'unique_active_building_series_count', 'unique_active_buildings_count', 'offers_in_sites_ratio'])
train = train.drop(columns=['offers_in_sites_ratio'])
# train['offer_count2'] = train['offer_count']**2
# train['rooms_offered_4_more_ratio2'] = train['rooms_offered_4_more_ratio']**2
train['total_area_avg2'] = train['total_area_avg']**2
###

In [15]:
# sns.pairplot(train.drop(columns='month'))

In [16]:
# усредняем train по quad-ам
train_avg_quad = train.drop(columns='month').groupby(['city_quadkey'], as_index=False).mean()
#.drop(columns='month')

In [17]:
# добавим усредненные во времени данные по quadkey из train 
test_trained = pd.merge(test, train_avg_quad,  how='left', left_on='city_quadkey', right_on = 'city_quadkey')

In [18]:
for i in range(len(test_trained.columns)):
    test_trained[test_trained.columns[i]] = test_trained[test_trained.columns[i]].fillna(np.mean(test_trained[test_trained.columns[i]]))

In [19]:
###
# test_trained = test_trained[['month', 'city_quadkey']]
###

In [20]:
np.sum(test_trained.isnull(), axis=0)
test_trained

,month,city_quadkey,apartment_ratio,unique_active_buildings_count,unique_active_building_series_count,rooms_offered_3_ratio,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,offer_count,rooms_offered_2_ratio,total_area_avg2
0,1.583010e+09,120132230323210,0.005712,8.125107,2.283418,0.246441,0.053182,0.320119,57.877019,0.017849,29.189027,0.377884,4080.539340
1,1.567285e+09,120132231322113,0.000000,3.343750,1.000000,0.692057,0.000000,0.198412,51.761600,0.000000,7.937500,0.109532,2693.218907
2,1.569877e+09,120132231322113,0.000000,3.343750,1.000000,0.692057,0.000000,0.198412,51.761600,0.000000,7.937500,0.109532,2693.218907
3,1.572556e+09,120132231322113,0.000000,3.343750,1.000000,0.692057,0.000000,0.198412,51.761600,0.000000,7.937500,0.109532,2693.218907
4,1.575148e+09,120132231322113,0.000000,3.343750,1.000000,0.692057,0.000000,0.198412,51.761600,0.000000,7.937500,0.109532,2693.218907
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38025,1.572556e+09,120311020100310,0.000000,5.750000,1.000000,0.014821,0.000000,0.105869,44.118595,0.000000,13.750000,0.879310,1959.491734
38026,1.575148e+09,120311020100310,0.000000,5.750000,1.000000,0.014821,0.000000,0.105869,44.118595,0.000000,13.750000,0.879310,1959.491734
38027,1.577826e+09,120311020100310,0.000000,5.750000,1.000000,0.014821,0.000000,0.105869,44.118595,0.000000,13.750000,0.879310,1959.491734
38028,1.580504e+09,120311020100310,0.000000,5.750000,1.000000,0.014821,0.000000,0.105869,44.118595,0.000000,13.750000,0.879310,1959.491734


In [21]:
# test_trained = pd.merge(test, train,  how='left', left_on=['city_quadkey','month'], right_on = ['city_quadkey','month'])
# нет пересечений

In [22]:
train_hb = pd.merge(train, hb_avg,  how='left', left_on='city_quadkey', right_on = 'city_quadkey')
test_hb = pd.merge(test_trained, hb_avg,  how='left', left_on='city_quadkey', right_on = 'city_quadkey')

In [23]:
train_hb

,apartment_ratio,unique_active_buildings_count,month,unique_active_building_series_count,rooms_offered_3_ratio,city_quadkey,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,...,groceries_and_everyday_items_cnt,art_cnt,site_id,healthcare_cnt,expect_demolition,longitude,ceiling_height,has_elevator,laundry_and_repair_services_cnt,build_year
0,0.0,1,1.517432e+09,1,0.000000,120132230303223,0.0,0.000000,44.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,35.611267,0.000000,0.0,1.0,0.000000
1,0.0,1,1.519852e+09,1,0.000000,120132230303223,0.0,0.000000,44.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,35.611267,0.000000,0.0,1.0,0.000000
2,0.0,1,1.527800e+09,1,0.000000,120132230332230,0.0,1.000000,34.500000,0.0,...,0.0,3.0,0.0,0.0,0.0,35.711987,0.000000,0.0,0.0,0.000000
3,0.0,1,1.530392e+09,1,0.000000,120132230332230,0.0,1.000000,34.500000,0.0,...,0.0,3.0,0.0,0.0,0.0,35.711987,0.000000,0.0,0.0,0.000000
4,0.0,1,1.483218e+09,1,1.000000,120132231322113,0.0,0.000000,56.000000,0.0,...,1.0,1.0,0.0,0.0,0.0,36.289875,0.588889,0.0,0.0,1320.888889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166022,0.0,9,1.554066e+09,1,0.052632,120311020100310,0.0,0.052632,46.421053,0.0,...,3.0,1.0,0.0,3.0,0.0,39.795544,0.489130,0.0,1.0,1116.043478
166023,0.0,9,1.556658e+09,1,0.047619,120311020100310,0.0,0.047619,46.309524,0.0,...,3.0,1.0,0.0,3.0,0.0,39.795544,0.489130,0.0,1.0,1116.043478
166024,0.0,9,1.559336e+09,1,0.043478,120311020100310,0.0,0.043478,45.995652,0.0,...,3.0,1.0,0.0,3.0,0.0,39.795544,0.489130,0.0,1.0,1116.043478
166025,0.0,10,1.561928e+09,1,0.041667,120311020100310,0.0,0.041667,45.933333,0.0,...,3.0,1.0,0.0,3.0,0.0,39.795544,0.489130,0.0,1.0,1116.043478


In [24]:
for i in range(len(train_hb.columns)):
    train_hb[train_hb.columns[i]] = train_hb[train_hb.columns[i]].fillna(np.mean(train_hb[train_hb.columns[i]]))
    test_hb[test_hb.columns[i]] = test_hb[test_hb.columns[i]].fillna(np.mean(test_hb[test_hb.columns[i]]))

In [25]:
np.sum(test_hb.isnull(), axis=0)

month                                  0
city_quadkey                           0
apartment_ratio                        0
unique_active_buildings_count          0
unique_active_building_series_count    0
rooms_offered_3_ratio                  0
rooms_offered_4_more_ratio             0
rooms_offered_1_ratio                  0
total_area_avg                         0
studio_ratio                           0
offer_count                            0
rooms_offered_2_ratio                  0
total_area_avg2                        0
beauty_cnt                             0
building_id                            0
building_series_id                     0
shopping_cnt                           0
flats_count                            0
latitude                               0
cafe_restaurant_eating_out_cnt         0
entertainment_cnt                      0
sport_cnt                              0
chain_cnt                              0
groceries_and_everyday_items_cnt       0
art_cnt         

In [26]:
test_hb['year'] = year_parser(test0['month'])
train_hb['year'] = year_parser(train0['month'])

test_hb['quarter'] = quarter_parser(test0['month'])
train_hb['quarter'] = quarter_parser(train0['month'])

test_hb['m'] = m_parser(test0['month'])
train_hb['m'] = m_parser(train0['month'])

In [27]:
# import seaborn as sns
# train_hb.corr()

In [28]:
# sns.heatmap(train_hb.corr())

In [29]:
train_hb['city_quadkey_1up'] = train_hb['city_quadkey']//10
train_hb['city_quadkey_2up'] = train_hb['city_quadkey']//10//10
train_hb['city_quadkey_3up'] = train_hb['city_quadkey']//10//10//10
train_hb['city_quadkey_4up'] = train_hb['city_quadkey']//10//10//10//10
train_hb['city_quadkey_5up'] = train_hb['city_quadkey']//10//10//10//10//10
train_hb['city_quadkey_6up'] = train_hb['city_quadkey']//10//10//10//10//10//10
train_hb['city_quadkey_7up'] = train_hb['city_quadkey']//10//10//10//10//10//10//10

test_hb['city_quadkey_1up'] = test_hb['city_quadkey']//10
test_hb['city_quadkey_2up'] = test_hb['city_quadkey']//10//10
test_hb['city_quadkey_3up'] = test_hb['city_quadkey']//10//10//10
test_hb['city_quadkey_4up'] = test_hb['city_quadkey']//10//10//10//10
test_hb['city_quadkey_5up'] = test_hb['city_quadkey']//10//10//10//10//10
test_hb['city_quadkey_6up'] = test_hb['city_quadkey']//10//10//10//10//10//10
test_hb['city_quadkey_7up'] = test_hb['city_quadkey']//10//10//10//10//10//10//10

In [30]:
len(set(train_hb['city_quadkey']//10//10//10//10//10//10//10))

12

In [31]:
len(set(train0['city_quadkey']))/4/4/4

110.890625

In [32]:
len(set(train0['city_quadkey']))
min(set(train0['city_quadkey'])), max(set(train0['city_quadkey']))
list(set(train0['city_quadkey']))[:20]

[120132323213322,
 120310101213200,
 120310101213201,
 120310013231122,
 120310101213202,
 120132323213332,
 120310101213203,
 120310101213210,
 120310101213211,
 120310101213212,
 120310101213213,
 120310101213220,
 120310101213221,
 120310101213222,
 120310101213223,
 120310101213230,
 120310101213231,
 120310101213232,
 120310101213233,
 120310103212110]

In [33]:
y_train

0         22727.273438
1         22727.273438
2         43478.261719
3         43478.261719
4         36250.000000
              ...     
166022    18744.729749
166023    19055.593378
166024    19081.275263
166025    19047.505127
166026    19047.505127
Name: avg_price_sqm, Length: 166027, dtype: float64

Данные росстат

https://rosstat.gov.ru/free_doc/new_site/prices/potr/tab-potr1.htm

In [34]:
i_neprod = pd.read_excel("I_prices_neprod.xlsx", sheet_name='Лист1')

In [35]:
i_neprod['chg']

val = [100]
for x in range(1, len(i_neprod['chg'])):
    val.append(val[x-1] * i_neprod['chg'][x] / 100)

i_neprod['i_neprod_val'] = val
i_neprod['i_neprod_chg'] = i_neprod['chg']
i_neprod = i_neprod.drop(columns='chg')

In [36]:
i_neprod.dtypes
#i_neprod[['chg', 'val']]

month             int64
year              int64
i_neprod_val    float64
i_neprod_chg    float64
dtype: object

In [37]:
train_hbi = pd.merge(train_hb, i_neprod,  how='left', left_on=['m', 'year'], right_on=['month', 'year']).drop(columns='month_y')
test_hbi = pd.merge(test_hb, i_neprod,  how='left', left_on=['m', 'year'], right_on=['month', 'year']).drop(columns='month_y')

In [38]:
test_hbi['month'] = test_hbi['month_x']
test_hbi = test_hbi.drop(columns=['month_x'])

train_hbi['month'] = train_hbi['month_x']
train_hbi = train_hbi.drop(columns=['month_x'])

#### Try adding the avg_price trend

In [39]:
train_avg_month = train0.groupby(['month'], as_index=False).mean()

train_avg_month['year'] = year_parser(train_avg_month['month'])
train_avg_month['m'] = m_parser(train_avg_month['month'])
train_avg_month = train_avg_month[['year', 'm', 'avg_price_sqm']]

train_avg_month['avg_p'] = train_avg_month['avg_price_sqm']
train_avg_month = train_avg_month.drop(columns='avg_price_sqm')

In [40]:
set(test0['month'])

{'2019-09-01',
 '2019-10-01',
 '2019-11-01',
 '2019-12-01',
 '2020-01-01',
 '2020-02-01',
 '2020-03-01'}

In [41]:
test_avg_month_nan_list = [[2019, 9, None],
                     [2019, 10, None],
                     [2019, 11, None],
                     [2019, 12, None],
                     [2020, 1, None],
                     [2020, 2, None],
                     [2020, 3, None]]

test_avg_month_nan = pd.DataFrame(test_avg_month_nan_list, columns=['year', 'm', 'avg_p'])

In [42]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_avg_month[['year', 'm']], train_avg_month['avg_p'])
test_avg_month_nan['avg_p'] = lr.predict(test_avg_month_nan[['year', 'm']])

In [43]:
test_avg_month_nan

,year,m,avg_p
0,2019,9,115354.970725
1,2019,10,115328.649456
2,2019,11,115302.328187
3,2019,12,115276.006918
4,2020,1,116549.602183
5,2020,2,116523.280914
6,2020,3,116496.959646


In [44]:
train_hbip = pd.merge(train_hbi, train_avg_month,  how='left', left_on=['m', 'year'], right_on=['m', 'year'])
test_hbip = pd.merge(test_hbi, test_avg_month_nan,  how='left', left_on=['m', 'year'], right_on=['m', 'year'])

MODEL

In [46]:
from catboost import CatBoostRegressor

In [47]:
cbr = CatBoostRegressor(silent=True)
cbr.fit(train_hbip, y_train)
cbr_pred1 = cbr.predict(test_hbip)

In [49]:
cbr_pred1

array([42010.27766289, 37972.11143087, 37979.21313855, ...,
       20567.85520095, 20548.9368274 , 20567.85520095])

In [50]:
test_subm1 = test0[['city_quadkey', 'month']]
test_subm1['avg_price_sqm'] = cbr_pred1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
test_subm1

,city_quadkey,month,avg_price_sqm
0,120132230323210,2020-03-01,42010.277663
1,120132231322113,2019-09-01,37972.111431
2,120132231322113,2019-10-01,37979.213139
3,120132231322113,2019-11-01,37979.213139
4,120132231322113,2019-12-01,37917.785587
...,...,...,...
38025,120311020100310,2019-11-01,19745.049783
38026,120311020100310,2019-12-01,19726.131410
38027,120311020100310,2020-01-01,20567.855201
38028,120311020100310,2020-02-01,20548.936827


In [52]:
test_subm1.to_csv('test_subm1.tsv', sep = '\t', index=False)

# ----------------------------------------------------------------------------------------